In [1]:
from datetime import datetime, timedelta
import pandas as pd

In [2]:
pre_tasks = [
    ('Sleep', '00:00', '07:00'),
    ('Sleep', '22:00', '23:59'),
    ('Pre-task', '13:00', '15:00'),
    ('Pre-task', '16:00', '17:00'),
    ('Pre-task', '18:00', '20:00'),
]

In [3]:
occupied_blocks = [(datetime.strptime(start, "%H:%M"), datetime.strptime(end, "%H:%M")) for _, start, end in pre_tasks]

schedule_records = []

# Add pre-scheduled and sleep blocks to schedule_records
for task_name, start, end in pre_tasks:
    schedule_records.append({
        'Task': task_name,
        'Start': start,
        'End': end
    })

In [4]:
def is_slot_free(slot_start, slot_end, occupied_blocks):
    return all(slot_start >= end or slot_end <= start for start, end in occupied_blocks)

In [5]:
def adjusted_deadline_diff(d1, d2, occupied_blocks):
    dt1 = datetime.strptime(d1, "%H:%M")
    dt2 = datetime.strptime(d2, "%H:%M")
    start_range = min(dt1, dt2)
    end_range = max(dt1, dt2)

    busy_hours = 0
    for start, end in occupied_blocks:
        if end <= start_range or start >= end_range:
            continue
        overlap_start = max(start, start_range)
        overlap_end = min(end, end_range)
        busy_hours += (overlap_end - overlap_start).seconds / 3600

    raw_diff = (end_range - start_range).seconds / 3600
    return round(max(raw_diff - busy_hours, 0), 2)

In [6]:
THRESHOLD_PRIORITY = 2
THRESHOLD_DEADLINE = 3

In [7]:
def greedy_assign_both_tasks(task1, task2, current_time_str, occupied_blocks, schedule_records):
    current_time = datetime.strptime(current_time_str, "%H:%M")
    tasks = sorted([task1, task2], key=lambda t: (-t['priority'], t['deadline']))
    for task in tasks:
        task['assigned'] = 0
        task['deadline_dt'] = datetime.strptime(task['deadline'], "%H:%M")

    t = datetime.strptime("00:00", "%H:%M")
    while t < datetime.strptime("23:59", "%H:%M"):
        slot_start = t
        slot_end = t + timedelta(hours=1)

        if slot_end <= current_time:
            t += timedelta(hours=1)
            continue

        for task in tasks:
            if task['assigned'] >= task['duration']:
                continue
            if slot_start >= current_time and slot_end <= task['deadline_dt']:
                if is_slot_free(slot_start, slot_end, occupied_blocks):
                    schedule_records.append({
                        'Task': task['name'],
                        'Start': slot_start.strftime("%H:%M"),
                        'End': slot_end.strftime("%H:%M")
                    })
                    occupied_blocks.append((slot_start, slot_end))
                    task['assigned'] += 1
                    break
        t += timedelta(hours=1)

    for task in tasks:
        if task['assigned'] < task['duration']:
            print(f"⚠️ {task['name']}: Only {task['assigned']}h scheduled, {task['duration'] - task['assigned']}h missing.")
        else:
            print(f"✅ {task['name']}: Fully scheduled.")

In [8]:
K1, K2, K3 = 10, 5, 3

In [9]:
def compute_weight(task, current_time_str):
    current = datetime.strptime(current_time_str, "%H:%M")
    deadline = datetime.strptime(task['deadline'], "%H:%M")
    deadline_gap = max((deadline - current).seconds / 3600, 0)
    
    return round(
        (task['priority'] * K1) +
        ((1 / (task['duration'] + 1)) * K2) +
        ((1 / (deadline_gap + 1)) * K3),
        3
    )

In [10]:
def split_and_weight_assign(task1, task2, current_time_str, occupied_blocks, schedule_records):
    # Compute weights
    task1['weight'] = compute_weight(task1, current_time_str)
    task2['weight'] = compute_weight(task2, current_time_str)
    
    print(f"\n🧠 Using Split & Weight Strategy")
    print(f"📌 {task1['name']} Weight: {task1['weight']}")
    print(f"📌 {task2['name']} Weight: {task2['weight']}")

    current_time = datetime.strptime(current_time_str, "%H:%M")
    deadline1 = datetime.strptime(task1['deadline'], "%H:%M")
    deadline2 = datetime.strptime(task2['deadline'], "%H:%M")
    final_deadline = max(deadline1, deadline2)

    # Sort tasks by weight
    tasks = sorted([task1, task2], key=lambda x: -x['weight'])
    for task in tasks:
        task['assigned'] = 0
        task['deadline_dt'] = datetime.strptime(task['deadline'], "%H:%M")

    # Loop through available slots
    t = datetime.strptime("00:00", "%H:%M")
    while t < datetime.strptime("23:59", "%H:%M"):
        slot_start = t
        slot_end = t + timedelta(hours=1)

        if slot_end <= current_time or slot_start >= final_deadline:
            t += timedelta(hours=1)
            continue

        for task in tasks:
            if task['assigned'] >= task['duration']:
                continue
            if slot_end <= task['deadline_dt']:
                if all(slot_start >= end or slot_end <= start for start, end in occupied_blocks):
                    # Assign split task block
                    schedule_records.append({
                        'Task': task['name'],
                        'Start': slot_start.strftime("%H:%M"),
                        'End': slot_end.strftime("%H:%M")
                    })
                    occupied_blocks.append((slot_start, slot_end))
                    task['assigned'] += 1
                    break  # move to next slot

        t += timedelta(hours=1)

    # Report summary
    for task in tasks:
        if task['assigned'] < task['duration']:
            print(f"⚠️ {task['name']}: Only {task['assigned']}h assigned, {task['duration'] - task['assigned']}h missing.")
        else:
            print(f"✅ {task['name']} fully scheduled by weight.")

In [11]:
def decide_strategy(task1, task2, current_time_str, occupied_blocks, schedule_records):
    priority_diff = abs(task1['priority'] - task2['priority'])
    deadline_diff = adjusted_deadline_diff(task1['deadline'], task2['deadline'], occupied_blocks)

    print(f"\n📌 Comparing {task1['name']} and {task2['name']}")
    print(f"🔢 Priority Difference: {priority_diff}")
    print(f"⏳ Adjusted Deadline Difference: {deadline_diff} hours")

    if priority_diff > THRESHOLD_PRIORITY:
        print("➡️ Use Greedy (Priority)")
        greedy_assign_both_tasks(task1, task2, current_time_str, occupied_blocks, schedule_records)
    elif deadline_diff > THRESHOLD_DEADLINE:
        print("➡️ Use Greedy (Deadline)")
        greedy_assign_both_tasks(task1, task2, current_time_str, occupied_blocks, schedule_records)
    else:
        print("➡️ Use Split & Weight ")
        split_and_weight_assign(task1, task2, current_time_str, occupied_blocks, schedule_records)

        

In [12]:
task1 = {'name': 'Task A', 'priority': 5, 'duration': 2, 'deadline': '14:00'}
task2 = {'name': 'Task B', 'priority': 4, 'duration': 4, 'deadline': '18:00'}
current_time_str = "09:00"

decide_strategy(task1, task2, current_time_str, occupied_blocks, schedule_records)

# Create final sorted DataFrame
schedule_df = pd.DataFrame(schedule_records).sort_values(by="Start").reset_index(drop=True)
schedule_df.to_csv("Final.csv", index=False)

print("\n✅ Final Schedule Saved to 'Final.csv'")
schedule_df


📌 Comparing Task A and Task B
🔢 Priority Difference: 1
⏳ Adjusted Deadline Difference: 2.0 hours
➡️ Use Split & Weight 

🧠 Using Split & Weight Strategy
📌 Task A Weight: 52.167
📌 Task B Weight: 41.3
✅ Task A fully scheduled by weight.
✅ Task B fully scheduled by weight.

✅ Final Schedule Saved to 'Final.csv'


,Task,Start,End
0,Sleep,00:00,07:00
1,Task A,09:00,10:00
2,Task A,10:00,11:00
3,Task B,11:00,12:00
4,Task B,12:00,13:00
5,Pre-task,13:00,15:00
6,Task B,15:00,16:00
7,Pre-task,16:00,17:00
8,Task B,17:00,18:00
9,Pre-task,18:00,20:00
